In [13]:
from katonic.ml import MyClient
import os
import numpy as np
from tensorflow import keras
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
myclient = MyClient()
mlflow = myclient.mlflow
client = myclient.client

In [10]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [28]:
mlflow.tensorflow.autolog()
exp_name = "mlflow-test-keras"
mlflow.set_experiment(exp_name)
exp_details = mlflow.get_experiment_by_name(exp_name)
with mlflow.start_run(run_name=exp_name):
    csv_url = (
        "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv"
    )
    data = pd.read_csv(csv_url, sep=";")
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    model = keras.Sequential(
        [
            keras.layers.Flatten(),
            keras.layers.Dense(30, activation="sigmoid"),
            keras.layers.Dense(24, activation="sigmoid"),
            keras.layers.Dense(6, activation="sigmoid"),
            keras.layers.Dense(1, activation="sigmoid"),
        ]
    )
    
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.fit(train_x, train_y, batch_size=3, epochs=5, validation_split=0.2)
    
    predictions = model.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predictions)
    
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    
    signature = mlflow.models.infer_signature(test_x, predictions)

    model_info = mlflow.tensorflow.log_model(model=model, artifact_path="model")
    

2023/10/24 10:01:54 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.
2023/10/24 10:01:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.9/site-packages/mlflow/tensorflow/_autolog.py:48: UserWarning: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data."


Epoch 1/5
  1/320 [..............................] - ETA: 3:55 - loss: 0.9823 - accuracy: 0.0000e+00WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0013s vs `on_train_batch_end` time: 0.0033s). Check your callbacks.


320/320 [==============================] - 1s 2ms/step - loss: -5.5856 - accuracy: 0.0000e+00 - val_loss: -9.4882 - val_accuracy: 0.0000e+00
Epoch 2/5
320/320 [==============================] - 0s 2ms/step - loss: -13.3490 - accuracy: 0.0000e+00 - val_loss: -17.5674 - val_accuracy: 0.0000e+00
Epoch 3/5
320/320 [==============================] - 1s 2ms/step - loss: -22.1120 - accuracy: 0.0000e+00 - val_loss: -25.7303 - val_accuracy: 0.0000e+00
Epoch 4/5
320/320 [==============================] - 0s 1ms/step - loss: -30.0827 - accuracy: 0.0000e+00 - val_loss: -33.3639 - val_accuracy: 0.0000e+00
Epoch 5/5
320/320 [==============================] - 0s 1ms/step - loss: -37.7593 - accuracy: 0.0000e+00 - val_loss: -40.8426 - val_accuracy: 0.0000e+00


2023/10/24 10:01:58 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: Cannot log input example or model signature for input with type <class 'pandas.core.frame.DataFrame'>. TensorFlow Keras autologging can only log input examples and model signatures for the following input types: numpy.ndarray, dict[string -> numpy.ndarray], tensorflow.keras.utils.Sequence, and tensorflow.data.Dataset (TensorFlow >= 2.1.0 required)


INFO:tensorflow:Assets written to: /tmp/tmpdrhvmrd6/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpdrhvmrd6/model/data/model/assets


13/13 [==============================] - 0s 1ms/step


INFO:tensorflow:Assets written to: /tmp/tmplesfu3sd/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmplesfu3sd/model/data/model/assets


12